<a href="https://colab.research.google.com/github/chiyeon01/ML_CONTEST/blob/main/DACON/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%20%EA%B3%A0%EA%B0%9D%20%EC%84%B8%EA%B7%B8%EB%A8%BC%ED%8A%B8%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/submit_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import clear_output

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMClassifier

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip '/content/gdrive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_신용카드 고객 세그먼트 분류_DACON/open.zip'

Archive:  /content/gdrive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_신용카드 고객 세그먼트 분류_DACON/open.zip
   creating: test/
   creating: test/1.회원정보/
  inflating: test/1.회원정보/201807_test_회원정보.parquet  
  inflating: test/1.회원정보/201808_test_회원정보.parquet  
  inflating: test/1.회원정보/201809_test_회원정보.parquet  
  inflating: test/1.회원정보/201810_test_회원정보.parquet  
  inflating: test/1.회원정보/201811_test_회원정보.parquet  
  inflating: test/1.회원정보/201812_test_회원정보.parquet  
   creating: test/2.신용정보/
  inflating: test/2.신용정보/201807_test_신용정보.parquet  
  inflating: test/2.신용정보/201808_test_신용정보.parquet  
  inflating: test/2.신용정보/201809_test_신용정보.parquet  
  inflating: test/2.신용정보/201810_test_신용정보.parquet  
  inflating: test/2.신용정보/201811_test_신용정보.parquet  
  inflating: test/2.신용정보/201812_test_신용정보.parquet  
   creating: test/3.승인매출정보/
  inflating: test/3.승인매출정보/201807_test_승인매출정보.parquet  
  inflating: test/3.승인매출정보/201808_test_승인매출정보.parquet  
  inflating: test/3.승인매출정보/201809_test_승인매출

In [ ]:
member_info_drop_list = ['이용카드수_체크_가족',
                         '이용금액_R3M_체크_가족',
                         '연회비할인카드수_B0M',
                         '할인금액_기본연회비_B0M',
                         '할인금액_제휴연회비_B0M'
                         ]

credit_info_drop_list = ['기준년월',
                         '시장연체상환여부_R3M',
                         '시장연체상환여부_R6M',
                         '시장단기연체여부_R3M'
                         ]

approve_info_drop_list = ['기준년월']

balance_info_drop_list = ['기준년월']

In [ ]:
# 1.
train_member_info_df07 = pd.read_parquet('/content/train/1.회원정보/201807_train_회원정보.parquet')
test_member_info_df07 = pd.read_parquet('/content/test/1.회원정보/201807_test_회원정보.parquet')
train_member_info_df07.drop(member_info_drop_list, axis=1, inplace=True)
test_member_info_df07.drop(member_info_drop_list, axis=1, inplace=True)

# 2.
train_credit_info_df07 = pd.read_parquet('/content/train/2.신용정보/201807_train_신용정보.parquet')
test_credit_info_df07 = pd.read_parquet('/content/test/2.신용정보/201807_test_신용정보.parquet')
train_credit_info_df07.drop(credit_info_drop_list, axis=1, inplace=True)
test_credit_info_df07.drop(credit_info_drop_list, axis=1, inplace=True)

# 3.
train_approve_info_df07 = pd.read_parquet('/content/train/3.승인매출정보/201807_train_승인매출정보.parquet')
test_approve_info_df07 = pd.read_parquet('/content/test/3.승인매출정보/201807_test_승인매출정보.parquet')
train_approve_info_df07.drop(approve_info_drop_list, axis=1, inplace=True)
test_approve_info_df07.drop(approve_info_drop_list, axis=1, inplace=True)

# 4.
train_claim_info_df07 = pd.read_parquet('/content/train/4.청구입금정보/201807_train_청구정보.parquet')
test_claim_info_df07 = pd.read_parquet('/content/test/4.청구입금정보/201807_test_청구정보.parquet')
train_claim_info_df07.drop('기준년월', axis=1, inplace=True)
test_claim_info_df07.drop('기준년월', axis=1, inplace=True)

# 5.
train_balance_info_df07 = pd.read_parquet('/content/train/5.잔액정보/201807_train_잔액정보.parquet')
test_balance_info_df07 = pd.read_parquet('/content/test/5.잔액정보/201807_test_잔액정보.parquet')
train_balance_info_df07.drop(balance_info_drop_list, axis=1, inplace=True)
test_balance_info_df07.drop(balance_info_drop_list, axis=1, inplace=True)

# 6.
train_channel_info_df07 = pd.read_parquet('/content/train/6.채널정보/201807_train_채널정보.parquet')
test_channel_info_df07 = pd.read_parquet('/content/test/6.채널정보/201807_test_채널정보.parquet')
train_channel_info_df07.drop('기준년월', axis=1, inplace=True)
test_channel_info_df07.drop('기준년월', axis=1, inplace=True)

# 7.
train_marketing_info_df07 = pd.read_parquet('/content/train/7.마케팅정보/201807_train_마케팅정보.parquet')
test_marketing_info_df07 = pd.read_parquet('/content/test/7.마케팅정보/201807_test_마케팅정보.parquet')
train_marketing_info_df07.drop('기준년월', axis=1, inplace=True)
test_marketing_info_df07.drop('기준년월', axis=1, inplace=True)

# 8.
train_result_info_df07 = pd.read_parquet('/content/train/8.성과정보/201807_train_성과정보.parquet')
test_result_info_df07 = pd.read_parquet('/content/test/8.성과정보/201807_test_성과정보.parquet')
train_result_info_df07.drop('기준년월', axis=1, inplace=True)
test_result_info_df07.drop('기준년월', axis=1, inplace=True)

In [ ]:
train_df = train_member_info_df07.merge(train_credit_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_approve_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_claim_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_balance_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_channel_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_marketing_info_df07, on='ID', how='outer')
train_df = train_df.merge(train_result_info_df07, on='ID', how='outer')

In [ ]:
# 30개의 컬럼이 null값이 존재함.
(train_df.isnull().sum(axis=0) > 0).sum()

30

In [ ]:
object_columns = train_df.dtypes[train_df.dtypes == 'object'].index.tolist()
for column in object_columns:
    if column == 'ID':
        continue
    encoder = LabelEncoder()
    train_df[column] = encoder.fit_transform(train_df[column])
    if column == 'Segment':
        class_to_cat = encoder.classes_

In [ ]:
train_df.drop(['ID', '기준년월'], axis=1, inplace=True)

In [ ]:
features = train_df.drop('Segment', axis=1, inplace=False)
targets = train_df['Segment']

In [ ]:
features.shape, targets.shape

((400000, 839), (400000,))

In [ ]:
lgb_clf = LGBMClassifier()
lgb_clf.fit(features, targets)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.948398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83122
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 718
[LightGBM] [Info] Start training from score -7.811623
[LightGBM] [Info] Start training from score -9.721166
[LightGBM] [Info] Start training from score -2.934402
[LightGBM] [Info] Start training from score -1.927459
[LightGBM] [Info] Start training from score -0.222075


LGBMClassifier()

In [ ]:
test_df = test_member_info_df07.merge(test_credit_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_approve_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_claim_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_balance_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_channel_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_marketing_info_df07, on='ID', how='outer')
test_df = test_df.merge(test_result_info_df07, on='ID', how='outer')

In [ ]:
object_columns = test_df.dtypes[test_df.dtypes == 'object'].index.tolist()
for column in object_columns:
    if column == 'ID':
        continue
    encoder = LabelEncoder()
    test_df[column] = encoder.fit_transform(test_df[column])

In [ ]:
features = test_df.drop(['ID', '기준년월'], axis=1, inplace=False)

In [ ]:
pred = lgb_clf.predict(features)

In [ ]:
pred

array([4, 4, 3, ..., 4, 2, 4])

In [ ]:
pred = class_to_cat[pred]

In [ ]:
submit_df = pd.read_csv('/content/sample_submission.csv')

In [ ]:
submit_df['Segment'] = pred

In [ ]:
submit_df

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,D
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C


In [ ]:
submit_df['Segment'].value_counts()

,count
Segment,
E,81289
D,13782
C,4652
A,234
B,43


In [ ]:
submit_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_신용카드 고객 세그먼트 분류_DACON/submit317(4).csv', index=False)

In [ ]:
train_df

,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,2,2,3,1,1,0,1,1,1,20130101,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,1,1,4,1,1,1,1,1,1,20170801,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,1,1,2,1,1,0,1,1,1,20080401,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,2,2,3,1,1,0,1,2,2,20160501,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,2,2,4,1,1,1,1,1,1,20180601,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,2,5,4,1,1,1,1,1,1,20010701,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
399996,2,3,3,1,1,1,1,1,1,20170701,...,0.999998,0.000000,0.999998,0.999998,0.924698,-0.449567,-0.513523,0.000000,1.951215,1.770262
399997,1,1,2,1,1,0,1,1,1,20090501,...,0.999998,0.305713,0.999998,0.999998,0.999998,0.580665,0.601561,0.000000,0.000000,0.000000
399998,1,2,4,1,1,1,1,1,1,20130101,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN


In [ ]:
####################################################################################
####################################################################################
#################################################################################### 경계선
####################################################################################
####################################################################################

In [ ]:
info_df = pd.read_parquet('/content/train/1.회원정보/201807_train_회원정보.parquet')

In [ ]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 78 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준년월                400000 non-null  int64  
 1   ID                  400000 non-null  object 
 2   남녀구분코드              400000 non-null  int64  
 3   연령                  400000 non-null  object 
 4   Segment             400000 non-null  object 
 5   회원여부_이용가능           400000 non-null  int64  
 6   회원여부_이용가능_CA        400000 non-null  int64  
 7   회원여부_이용가능_카드론       400000 non-null  int64  
 8   소지여부_신용             400000 non-null  int64  
 9   소지카드수_유효_신용         400000 non-null  int64  
 10  소지카드수_이용가능_신용       400000 non-null  int64  
 11  입회일자_신용             400000 non-null  int64  
 12  입회경과개월수_신용          400000 non-null  int64  
 13  회원여부_연체             400000 non-null  int64  
 14  이용거절여부_카드론          400000 non-null  int64  
 15  동의여부_한도증액안내         400000 non-nul

In [ ]:
object_columns = info_df.dtypes[info_df.dtypes == 'object'].index.tolist()
for column in object_columns:
    if column == 'ID':
        continue
    encoder = LabelEncoder()
    info_df[column] = encoder.fit_transform(info_df[column])
    if column == 'Segment':
        class_to_cat = encoder.classes_

In [ ]:
info_df.drop(['ID', '기준년월'], axis=1, inplace=True)

In [ ]:
features = info_df.drop('Segment', axis=1, inplace=False)
targets = info_df['Segment']

In [ ]:
features.shape, targets.shape

((400000, 75), (400000,))

In [ ]:
lgb_clf = LGBMClassifier()
lgb_clf.fit(features, targets)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3112
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 64
[LightGBM] [Info] Start training from score -7.811623
[LightGBM] [Info] Start training from score -9.721166
[LightGBM] [Info] Start training from score -2.934402
[LightGBM] [Info] Start training from score -1.927459
[LightGBM] [Info] Start training from score -0.222075


LGBMClassifier()

In [ ]:
test_df = pd.read_parquet('/content/test/1.회원정보/201807_test_회원정보.parquet')
test_df

,기준년월,ID,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TEST_00000,1,40대,1,1,0,1,2,2,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),45
1,201807,TEST_00001,1,60대,1,1,0,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀독립기,19
2,201807,TEST_00002,1,40대,1,1,1,1,2,2,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),5
3,201807,TEST_00003,2,40대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),7
4,201807,TEST_00004,2,40대,1,0,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,201807,TEST_99995,2,60대,0,0,0,1,1,0,...,0,0,0,0개,0개,0개,0개,0,노년생활,21
99996,201807,TEST_99996,1,30대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,7
99997,201807,TEST_99997,2,30대,1,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),6
99998,201807,TEST_99998,1,30대,1,1,1,1,3,3,...,0,0,0,0개,0개,0개,0개,1,가족구축기,1


In [ ]:
object_columns = test_df.dtypes[test_df.dtypes == 'object'].index.tolist()
for column in object_columns:
    if column == 'ID':
        continue
    encoder = LabelEncoder()
    test_df[column] = encoder.fit_transform(test_df[column])

In [ ]:
features = test_df.drop(['ID', '기준년월'], axis=1, inplace=False)

In [ ]:
pred = lgb_clf.predict(features)

In [ ]:
pred

array([4, 4, 4, ..., 4, 2, 4])

In [ ]:
pred = class_to_cat[pred]

In [ ]:
submit_df = pd.read_csv('/content/sample_submission.csv')

In [ ]:
submit_df['Segment'] = pred

In [ ]:
submit_df

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C


In [ ]:
submit_df['Segment'].value_counts()

,count
Segment,
E,84581
D,11729
C,3537
A,89
B,64


In [ ]:
submit_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/z.대회 & 프로젝트/2025_신용카드 고객 세그먼트 분류_DACON/test.csv', index=False)

In [ ]:
'''
2025.03.15
Segment = target값. => 0.387

=======================================================================================================================================================

2025.03.16
ID를 기준으로 합했더니 총 30개의 null값이 포함된 column이 나옴.
총 컬럼 수는 857개임.(ID, 날짜 포함) => 0.476
회원정보에서 데이터의 값이 모두 같은 경우가 있음. 이를 drop함. => 0.480

=======================================================================================================================================================

2025.03.17
신용정보에서 데이터의 값이 모두 같은 경우 or 테스트 데이터에는 아예 없는 값이 학습 데이터에는 존재하는 경우가 있음. 이를 drop함. => 0.519
승인매출과 잔액정보에서도 위와 비슷한 값을 drop했으나, 성능이 떨어짐.
예상하건데 같은 값만 나오는 것들을 지운 경우 성능이 떨어진 경우는 '돈'에 관련된 것임(적어도 현재까지)
반면에, 성능이 오른 경우는 회원정보, 신용정보와 같이 돈과 직접적인 관련이 없는 경우임(수치상)
따라서 채널정보, 마케팅정보에서는 또 성능 효과가 발생할 가능성이 있으므로 내일 해보는게 좋을듯?
=======================================================================================================================================================

'''

'\n2025.03.15\nSegment = target값. => 0.387\n\n2025.03.16\nID를 기준으로 합했더니 총 30개의 null값이 포함된 column이 나옴.\n총 컬럼 수는 857개임.(ID, 날짜 포함) => 0.476\n'